In [31]:
# 기본패키지

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

# 모델
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# 모델
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
#pd.set_option('display.max_columns', None) # 컬럼 수가 많으므로 요약되어 표시되지 않게 함
import warnings
warnings.filterwarnings(action='ignore')

In [32]:
rf_rg = RandomForestRegressor()
xgb_rg = XGBRegressor()
lgbm_rg = LGBMRegressor()
cat_rg = CatBoostRegressor(verbose=0)

In [298]:
barrel_df = pd.read_csv("./외국인타자MLB배럴성적2.csv")
minor_df = pd.read_csv("./외국인타자_마이너성적.csv")
kbo_df = pd.read_csv("./외국인타자_국내성적.csv")
kbo_df = kbo_df[["이름","재계약","타율","OPS","WAR"]]
kbo_df = kbo_df.rename(columns = {"OPS" : "KBO_OPS", "WAR" : "KBO_WAR"})


# 배럴타구로 예측

In [299]:
barrel_df.head(1)

,영문이름,이름,구단,재계약,Season,Age,Pitches,Batted Balls,Barrels,Barrel%,...,Launch Angle,Sweet Spot %,XBA,XSLG,WOBA,XWOBA,XWOBACON,HardHit%,K%,BB%
0,Daniel Casey Dorn,대니 돈,키움,1.0,2015,30,125,20,0,0.0,...,8.6,25.0,0.184,0.234,0.192,0.215,0.279,35.0,31.3,6.3


In [300]:
merge_df = pd.merge(barrel_df,kbo_df,on=["이름"])
merge_df = merge_df.rename(columns={"재계약_x" : "재계약"})

In [301]:
merge_df.head(1)

,영문이름,이름,구단,재계약,Season,Age,Pitches,Batted Balls,Barrels,Barrel%,...,WOBA,XWOBA,XWOBACON,HardHit%,K%,BB%,재계약_y,타율,KBO_OPS,KBO_WAR
0,Daniel Casey Dorn,대니 돈,키움,1.0,2015,30,125,20,0,0.0,...,0.192,0.215,0.279,35.0,31.3,6.3,1,0.295,0.893,3.03


In [302]:
merge_df.shape

(45, 27)

In [303]:
columns = barrel_df.columns
columns
X_data = merge_df[columns].drop(columns = ["영문이름","이름","구단","Season","재계약"])
y_data = merge_df["타율"]

cross_val_score(cat_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-0.05318290712994898

In [304]:
cross_val_predict(cat_rg,X_data,y_data)

array([0.25790139, 0.27303013, 0.26867593, 0.26404124, 0.26404124,
       0.30737647, 0.2822558 , 0.25655216, 0.25722186, 0.26333055,
       0.28269746, 0.23444463, 0.27108092, 0.25245513, 0.25570653,
       0.26201492, 0.23454896, 0.24005208, 0.2873909 , 0.2655583 ,
       0.2885132 , 0.28628814, 0.27325434, 0.26617392, 0.29444813,
       0.25448691, 0.26724151, 0.25397276, 0.23531712, 0.23248805,
       0.21123356, 0.21123356, 0.28412165, 0.28860561, 0.24071083,
       0.25349275, 0.2639905 , 0.26590084, 0.26183992, 0.27025116,
       0.23687551, 0.223969  , 0.25557896, 0.24102812, 0.20791387])

In [307]:
temp = merge_df[["이름","구단","재계약","타율"]]
temp["predict"] = cross_val_predict(xgb_rg,X_data,y_data)
temp["차이"] = abs(temp["타율"] - temp["predict"])
temp

,이름,구단,재계약,타율,predict,차이
0,대니 돈,키움,1.0,0.295,0.251646,0.043354
1,헥터 고메즈,SSG,0.0,0.283,0.299749,0.016749
2,윌린 로사리오,한화,1.0,0.339,0.283144,0.055856
3,저스틴 맥스웰,롯데,0.0,0.288,0.222788,0.065212
4,재비어 스크럭스,NC,1.0,0.300,0.222788,0.077212
5,제임스 로니,LG,0.0,0.278,0.338326,0.060326
6,대니 워스,SSG,0.0,0.111,0.335501,0.224501
7,제이미 로맥,SSG,1.0,0.242,0.258052,0.016052
8,다린 러프,삼성,1.0,0.315,0.204909,0.110091
9,지미 파레디스,두산,0.0,0.138,0.291606,0.153606


In [310]:
X_data.head(1)

,Age,Pitches,Batted Balls,Barrels,Barrel%,Barrel/PA,Exit Velocity,Max EV,Launch Angle,Sweet Spot %,XBA,XSLG,WOBA,XWOBA,XWOBACON,HardHit%,K%,BB%
0,30,125,20,0,0.0,0.0,86.2,104.7,8.6,25.0,0.184,0.234,0.192,0.215,0.279,35.0,31.3,6.3


In [313]:
kbo_df

,이름,재계약,타율,KBO_OPS,KBO_WAR
0,야마이코 나바로,1,0.308,0.969,6.23
1,호르헤 칸투,0,0.309,0.899,3.10
2,조쉬 벨,0,0.267,0.779,1.47
3,브래드 스나이더,0,0.210,0.692,0.14
4,비니 로티노,0,0.306,0.805,1.11
...,...,...,...,...,...
72,닉 마티니,0,0.300,0.833,3.96
73,DJ 피터스,0,0.228,0.701,1.18
74,잭 렉스,1,0.330,0.906,2.30
75,소크라테스 브리토,1,0.311,0.848,4.97


In [325]:
xgb_rg = XGBRegressor()

avg_df = pd.merge(barrel_df, kbo_df, on = ["이름"])
#avg_df.head(1)
X_train = avg_df.drop(columns = ["영문이름","이름","구단","재계약_x","Season","재계약_y","타율","KBO_OPS","KBO_WAR"])
y_train = avg_df["타율"]

xgb_rg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [326]:
X_train.shape

(45, 18)

In [327]:

X_test = barrel_df.iloc[-5:]
X_test = X_test.drop(columns=["영문이름","이름","구단","재계약","Season"])

xgb_rg.predict(X_test)

array([0.2934329 , 0.27012524, 0.26750162, 0.23705831, 0.22310537],
      dtype=float32)

In [328]:
xgb_rg = XGBRegressor()

avg_df = pd.merge(barrel_df, kbo_df, on = ["이름"])
#avg_df.head(1)
X_train = avg_df.drop(columns = ["영문이름","이름","구단","재계약_x","Season","재계약_y","타율","KBO_OPS","KBO_WAR"])
y_train = avg_df["KBO_OPS"]

xgb_rg.fit(X_train,y_train)
X_test = barrel_df.iloc[-5:]
X_test = X_test.drop(columns=["영문이름","이름","구단","재계약","Season"])

xgb_rg.predict(X_test)

array([0.92127967, 0.82926595, 0.84734744, 0.83148515, 0.81477886],
      dtype=float32)

In [329]:
xgb_rg = XGBRegressor()

avg_df = pd.merge(barrel_df, kbo_df, on = ["이름"])
#avg_df.head(1)
X_train = avg_df.drop(columns = ["영문이름","이름","구단","재계약_x","Season","재계약_y","타율","KBO_OPS","KBO_WAR"])
y_train = avg_df["KBO_WAR"]

xgb_rg.fit(X_train,y_train)
X_test = barrel_df.iloc[-5:]
X_test = X_test.drop(columns=["영문이름","이름","구단","재계약","Season"])

xgb_rg.predict(X_test)

array([2.90986   , 0.15578642, 2.835956  , 0.72054744, 1.3467414 ],
      dtype=float32)

In [335]:
kbo_df[kbo_df["재계약"] == 1].describe()

,재계약,타율,KBO_OPS,KBO_WAR
count,27.0,27.000000,27.000000,27.000000
mean,1.0,0.308370,0.939778,3.834444
std,0.0,0.026448,0.102218,1.867961
min,1.0,0.242000,0.778000,1.050000
25%,1.0,0.297500,0.881500,2.500000
50%,1.0,0.308000,0.911000,3.720000
75%,1.0,0.314500,0.972000,4.770000
max,1.0,0.381000,1.288000,10.710000


In [142]:
merge_df.head(1)

,영문이름,이름,구단,재계약,Season,Age,Pitches,Batted Balls,Barrels,Barrel%,...,WOBA,XWOBA,XWOBACON,HardHit%,K%,BB%,재계약_y,타율,KBO_OPS,KBO_WAR
0,Daniel Casey Dorn,대니 돈,키움,1.0,2015,30,125,20,0,0.0,...,0.192,0.215,0.279,35.0,31.3,6.3,1.0,0.295,0.893,3.03


In [73]:
columns = barrel_df.columns

X_data = merge_df[columns].drop(columns = ["영문이름","이름","구단","재계약","Season"])
y_data = merge_df["OPS"]

cross_val_score(lgbm_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-0.1476371300116842

In [74]:
columns = barrel_df.columns

X_data = merge_df[columns].drop(columns = ["영문이름","이름","구단","재계약","Season"])
y_data = merge_df["WAR"]

cross_val_score(lgbm_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-1.5125110822040038

In [75]:
minor_df

,연도,구단,이름,재계약,G,PA,AB,R,H,2B,...,BA,OBP,SLG,OPS,TB,GDP,HBP,SH,SF,IBB
0,2014,삼성,야마이코 나바로,1,108,452,390,59,104,21,...,0.267,0.354,0.418,0.772,163,7,3,0,6,0
1,2014,두산,호르헤 칸투,0,21,90,86,12,25,5,...,0.291,0.300,0.488,0.788,42,5,0,0,2,0
2,2014,LG,조쉬 벨,0,52,210,177,25,40,8,...,0.226,0.324,0.356,0.680,63,6,0,0,5,1
3,2014,LG,브래드 스나이더,0,116,458,411,77,128,29,...,0.311,0.370,0.494,0.864,203,6,1,1,5,2
4,2014,넥센,비니 로티노,0,96,420,374,60,111,28,...,0.297,0.355,0.455,0.809,170,5,4,0,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2022,NC,닉 마티니,0,78,323,270,56,72,9,...,0.267,0.387,0.444,0.831,120,6,11,0,0,1
72,2022,롯데,DJ 피터스,0,50,204,180,24,42,9,...,0.233,0.319,0.372,0.691,67,2,3,0,1,0
73,2022,롯데,잭 렉스,1,87,377,321,69,90,24,...,0.280,0.382,0.539,0.921,173,8,8,0,2,0
74,2022,KIA,소크라테스 브리토,1,107,419,375,57,94,16,...,0.251,0.315,0.376,0.691,141,5,6,0,6,0


# minor 성적으로 예측

In [79]:
merge_df = pd.merge(minor_df,kbo_df,on=["이름","재계약"])
merge_df

,연도,구단,이름,재계약,G,PA,AB,R,H,2B,...,OPS,TB,GDP,HBP,SH,SF,IBB,타율,KBO_OPS,KBO_WAR
0,2014,삼성,야마이코 나바로,1,108,452,390,59,104,21,...,0.772,163,7,3,0,6,0,0.308,0.969,6.23
1,2014,두산,호르헤 칸투,0,21,90,86,12,25,5,...,0.788,42,5,0,0,2,0,0.309,0.899,3.10
2,2014,LG,조쉬 벨,0,52,210,177,25,40,8,...,0.680,63,6,0,0,5,1,0.267,0.779,1.47
3,2014,LG,브래드 스나이더,0,116,458,411,77,128,29,...,0.864,203,6,1,1,5,2,0.210,0.692,0.14
4,2014,넥센,비니 로티노,0,96,420,374,60,111,28,...,0.809,170,5,4,0,8,5,0.306,0.805,1.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2022,삼성,호세 피렐라,1,88,372,342,69,112,22,...,0.973,204,2,3,0,2,4,0.286,0.855,3.26
65,2021,한화,라이온 힐리,0,88,372,342,69,112,22,...,0.973,204,2,3,0,2,4,0.257,0.700,0.35
66,2021,한화,에르난 페레즈,0,27,121,107,18,31,10,...,0.895,56,4,0,0,0,0,0.267,0.730,0.69
67,2022,롯데,DJ 피터스,0,50,204,180,24,42,9,...,0.691,67,2,3,0,1,0,0.228,0.701,1.18


In [81]:
columns = minor_df.columns

X_data = merge_df[columns].drop(columns = ["이름","구단","재계약"])
y_data = merge_df["타율"]

cross_val_score(lgbm_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-0.048268716483224436

In [85]:
columns = minor_df.columns

X_data = merge_df[columns].drop(columns = ["이름","구단","재계약"])
y_data = merge_df["KBO_OPS"]

cross_val_score(cat_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-0.12767322408693083

In [88]:
columns = minor_df.columns

X_data = merge_df[columns].drop(columns = ["이름","구단","재계약"])
y_data = merge_df["KBO_WAR"]

cross_val_score(lgbm_rg,X_data,y_data,scoring="neg_mean_absolute_error").mean()

-1.4328171620038315

# 얼굴 PCA

In [155]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4) # 주성분을 몇개로 할지 결정



In [156]:
face_dataset = pd.read_csv("./타자얼굴사진_오렌지임베딩.csv")
face_dataset2 = pd.read_csv("./타자얼굴사진_오렌지임베딩_라벨수정.csv")

In [157]:
data = face_dataset.drop(columns = ["Unnamed: 2", "size", "width", "height","재계약","이름"])
data2 = face_dataset2.drop(columns = ["Unnamed: 2", "size", "width", "height","재계약","이름"])

In [158]:
data

,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,...,Unnamed: 2044,Unnamed: 2045,Unnamed: 2046,Unnamed: 2047,Unnamed: 2048,Unnamed: 2049,Unnamed: 2050,Unnamed: 2051,Unnamed: 2052,Unnamed: 2053
0,0.144235,0.214922,0.000507,0.230852,0.055338,0.190327,0.422432,0.323068,0.632708,0.540388,...,0.110022,0.019359,0.097470,1.398580,1.140330,1.470890,0.131892,0.002172,0.000000,0.087585
1,0.375483,0.186434,0.038951,0.376486,0.048386,0.078989,0.587438,0.203096,0.480619,0.377400,...,0.391663,0.469408,0.059494,1.241490,0.561510,0.789311,0.089493,0.020344,0.089230,0.080406
2,0.206742,0.352354,0.000000,0.319393,0.129964,0.226437,0.528457,0.230924,0.253103,0.857314,...,0.438406,0.191309,0.273648,0.632021,0.341713,0.978627,0.189920,0.062408,0.004243,0.184213
3,0.178468,0.242643,0.003908,0.319384,0.162789,0.703334,0.492763,0.239284,0.608889,0.693923,...,0.888367,0.226742,0.372235,1.203300,0.862949,0.618488,0.034147,0.183850,0.000000,0.088913
4,0.279750,0.123999,0.066130,0.330183,0.089147,0.239294,0.818451,0.541217,0.174553,0.296911,...,0.529637,0.382231,0.240935,1.352470,0.483653,0.957445,0.001848,0.094242,0.019609,0.374478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.279966,0.245019,0.034286,0.242645,0.023046,0.089658,0.617326,0.177323,0.406420,0.547843,...,0.127100,0.478069,0.196349,1.074790,0.871169,1.217630,0.412305,0.000000,0.000000,0.227275
73,0.295471,0.207789,0.043837,0.179706,0.181316,0.459203,0.629376,0.434749,0.251177,0.703419,...,0.544242,0.304189,0.310420,1.079540,0.470909,0.947180,0.114960,0.012160,0.000000,0.310165
74,0.127145,0.068625,0.059579,0.570567,0.146204,0.341086,1.112030,0.411414,0.824074,1.016550,...,0.241656,0.856658,0.281153,1.242930,0.226189,0.855392,0.067820,0.329389,0.087977,0.227984
75,0.139360,0.094605,0.000000,0.178610,0.180731,0.394039,0.370310,0.288621,0.539865,0.657078,...,0.161003,0.071225,0.051891,0.840497,1.358740,1.130330,0.096857,0.054398,0.000035,0.616415


In [173]:
pca_df = pd.DataFrame(pca.fit_transform(data), columns=["PCA1","PCA2","PCA3","PCA4"])
pca_df

,PCA1,PCA2,PCA3,PCA4
0,1.284872,-0.336774,0.393235,-1.273952
1,-2.844535,0.771240,0.357858,-3.278572
2,1.460736,1.700855,0.040147,-1.209631
3,2.655309,3.578816,-0.855950,-0.005881
4,0.008052,-0.672386,-1.412765,0.307356
...,...,...,...,...
72,-0.173532,0.519327,-1.018806,-0.670637
73,3.361797,3.820105,-0.203604,2.385811
74,2.681150,2.303731,-1.455868,3.116756
75,3.926896,-4.533700,0.384252,-1.636684


In [168]:
pca.explained_variance_ratio_

array([0.10449618, 0.09533372, 0.06565112, 0.0616345 ])

In [174]:
pca_df["이름"] = face_dataset["이름"]
pca_df["재계약"] = face_dataset["재계약"]

In [175]:
pca_df

,PCA1,PCA2,PCA3,PCA4,이름,재계약
0,1.284872,-0.336774,0.393235,-1.273952,DJ 피터스,0
1,-2.844535,0.771240,0.357858,-3.278572,나이저 모건,0
2,1.460736,1.700855,0.040147,-1.209631,닉 마티니,0
3,2.655309,3.578816,-0.855950,-0.005881,다니엘 팔카,0
4,0.008052,-0.672386,-1.412765,0.307356,대니 워스,0
...,...,...,...,...,...,...
72,-0.173532,0.519327,-1.018806,-0.670637,제이미 로맥,1
73,3.361797,3.820105,-0.203604,2.385811,짐 아두치,1
74,2.681150,2.303731,-1.455868,3.116756,프레스턴 터커,1
75,3.926896,-4.533700,0.384252,-1.636684,호세 미겔 페르난데스,1


In [179]:
face_pca_df = pd.merge(pca_df, barrel_df, on = ["이름"])

In [181]:
face_pca_df.columns

Index(['PCA1', 'PCA2', 'PCA3', 'PCA4', '이름', '재계약_x', '영문이름', '구단', '재계약_y',
       'Season', 'Age', 'Pitches', 'Batted Balls', 'Barrels', 'Barrel%',
       'Barrel/PA', 'Exit Velocity', 'Max EV', 'Launch Angle', 'Sweet Spot %',
       'XBA', 'XSLG', 'WOBA', 'XWOBA', 'XWOBACON', 'HardHit%', 'K%', 'BB%'],
      dtype='object')

In [275]:
X_data = face_pca_df.drop(columns = ["이름","재계약_x","영문이름","구단","재계약_y","Season"])
y_data = face_pca_df["재계약_y"]

In [276]:
X_ss = ss.fit_transform(X_data)

In [186]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

kfold = KFold(n_splits=5, shuffle=True,random_state=2732)
xgb = XGBClassifier(tree_method = "gpu_hist")
cat = CatBoostClassifier(verbose = 0,task_type="GPU")
lgbm = LGBMClassifier()
lr = LogisticRegression()
svc = SVC()


In [232]:
(0.56+0.56+0.45+0.67+0.75)/5

0.5980000000000001

In [264]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.13333333333333333

In [281]:
cross_val_score(svc, X_ss, y_data, cv=kfold, scoring = "roc_auc").mean()

0.4776190476190476

In [274]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.0

In [187]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

array([0.55555556, 0.55555556, 0.44444444, 0.66666667, 0.75      ])

In [224]:
cross_val_score(lr, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.55

In [225]:
cross_val_score(lgbm, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.6611111111111112

In [226]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.6611111111111112

In [233]:
cross_val_score(xgb, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.5027777777777778

In [188]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

array([0.28571429, 0.4       , 0.05      , 0.71428571, 0.83333333])

In [227]:
cross_val_score(xgb, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.27904761904761904

In [228]:
cross_val_score(lgbm, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.5

In [229]:
cross_val_score(lr, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.5723809523809524

In [231]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.5219047619047619

In [189]:
(0.29+0.4+0.05+0.71+0.83)/5

0.45599999999999996

# KBO + 얼굴

In [288]:
kbo_df = pd.read_csv("./외국인타자_국내성적.csv")
#kbo_df = kbo_df[["이름","재계약","타율","OPS","WAR"]]
kbo_df = kbo_df.rename(columns = {"OPS" : "KBO_OPS", "WAR" : "KBO_WAR"})

In [289]:
kbo_df.head(1)

,연도,구단,KBO_팀순위,이름,수비율,연봉,재계약,나이,P,G,...,병살,희타,희비,타율,출루,장타,KBO_OPS,wOBA,wRC+,KBO_WAR
0,2014,삼성,1.0,야마이코 나바로,0.987,25000.0,1,27,2B,125,...,7,0,5,0.308,0.417,0.552,0.969,0.424,139.2,6.23


In [290]:
temp = kbo_df.copy()
temp_data = temp.drop(columns = ["연도","구단","KBO_팀순위","이름","수비율","연봉","재계약","P"])
temp_y = temp["재계약"]

In [291]:
temp_y

0     1
1     0
2     0
3     0
4     0
     ..
72    0
73    0
74    1
75    1
76    0
Name: 재계약, Length: 77, dtype: int64

In [292]:
temp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   나이       77 non-null     int64  
 1   G        77 non-null     int64  
 2   타석       77 non-null     int64  
 3   타수       77 non-null     int64  
 4   득점       77 non-null     int64  
 5   안타       77 non-null     int64  
 6   2타       77 non-null     int64  
 7   3타       77 non-null     int64  
 8   홈런       77 non-null     int64  
 9   루타       77 non-null     int64  
 10  타점       77 non-null     int64  
 11  도루       77 non-null     int64  
 12  도실       77 non-null     int64  
 13  볼넷       77 non-null     int64  
 14  사구       77 non-null     int64  
 15  고4       77 non-null     int64  
 16  삼진       77 non-null     int64  
 17  병살       77 non-null     int64  
 18  희타       77 non-null     int64  
 19  희비       77 non-null     int64  
 20  타율       77 non-null     float64
 21  출루       77 non-nu

In [293]:
cross_val_score(cat, temp_data, temp_y, cv=kfold, scoring = "f1").mean()

0.5587114845938375

In [294]:
cross_val_score(xgb, temp_data, temp_y, cv=kfold, scoring = "f1").mean()

0.6927807486631016

In [295]:
cross_val_score(lgbm, temp_data, temp_y, cv=kfold, scoring = "f1").mean()

0.6524364524364524

In [296]:
cross_val_score(lr, temp_data, temp_y, cv=kfold, scoring = "f1").mean()

0.7406162464985994

In [297]:
cross_val_score(svc, temp_data, temp_y, cv=kfold, scoring = "f1").mean()

0.7268270944741533

In [246]:
kbo_face = pd.merge(kbo_df, pca_df, on = "이름")

In [247]:
kbo_face.columns

Index(['연도', '구단', 'KBO_팀순위', '이름', '수비율', '연봉', '재계약_x', '나이', 'P', 'G', '타석',
       '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구',
       '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'KBO_OPS', 'wOBA',
       'wRC+', 'KBO_WAR', 'PCA1', 'PCA2', 'PCA3', 'PCA4', '재계약_y'],
      dtype='object')

In [248]:
kbo_face_df = kbo_face.drop(columns = ["연도","구단","KBO_팀순위","이름","수비율","연봉","재계약_x","P"])
kbo_face_df

,나이,G,타석,타수,득점,안타,2타,3타,홈런,루타,...,장타,KBO_OPS,wOBA,wRC+,KBO_WAR,PCA1,PCA2,PCA3,PCA4,재계약_y
0,27,125,602,500,118,154,27,1,31,276,...,0.552,0.969,0.424,139.2,6.23,-2.991404,1.397617,-0.645810,1.118868,1
1,32,111,419,376,56,116,25,1,18,197,...,0.524,0.899,0.395,134.8,3.10,-0.201155,1.617601,-1.449454,0.591825,0
2,28,63,275,240,33,64,8,1,10,104,...,0.433,0.779,0.347,101.6,1.47,-4.085174,0.437642,-0.733083,-0.409265,0
3,32,37,113,100,17,21,7,0,4,40,...,0.400,0.692,0.308,76.2,0.14,1.356951,-0.080244,-0.573521,0.349969,0
4,34,79,247,216,30,66,18,0,2,90,...,0.417,0.805,0.368,104.1,1.11,2.283241,4.732939,-0.738762,0.741525,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,32,139,576,510,67,153,34,1,16,237,...,0.465,0.833,0.376,134.9,3.96,1.460736,1.700855,0.040147,-1.209631,0
72,26,85,354,316,32,72,16,0,13,127,...,0.402,0.701,0.319,93.7,1.18,1.284872,-0.336774,0.393235,-1.273952,0
73,29,56,251,218,32,72,10,1,8,108,...,0.495,0.906,0.411,156.3,2.30,1.011088,0.236652,1.328121,-2.532381,1
74,30,127,554,514,83,160,29,7,17,254,...,0.494,0.848,0.380,137.0,4.97,1.717370,1.666663,-2.849031,-2.173165,1


In [249]:
X_data = kbo_face_df.drop(columns = ["재계약_y"])
y_data = kbo_face_df["재계약_y"]

In [251]:
ss = StandardScaler()
ss.fit(X_data)
X_ss = ss.transform(X_data)

In [250]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.6946608946608948

In [254]:
cross_val_score(xgb, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.6577777777777778

In [255]:
cross_val_score(lgbm, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.655600539811066

In [258]:
cross_val_score(lr, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.7008547008547008

In [262]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "f1").mean()

0.6248313090418354

In [221]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.8944083694083694

In [222]:
cross_val_score(cat, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.8666666666666668

In [236]:
cross_val_score(xgb, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.8666666666666666

In [237]:
cross_val_score(lgbm, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.8666666666666666

In [238]:
cross_val_score(lr, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.8133333333333332

In [239]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "accuracy").mean()

0.8266666666666665

In [240]:
cross_val_score(xgb, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.9340187590187592

In [241]:
cross_val_score(lgbm, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.9315536315536315

In [242]:
cross_val_score(lr, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.9310966810966811

In [244]:
cross_val_score(svc, X_data, y_data, cv=kfold, scoring = "roc_auc").mean()

0.7522607022607023